# Computer Vision Nanodegree

## Project: Image Captioning

---

In this notebook, you will train your CNN-RNN model.  

You are welcome and encouraged to try out many different architectures and hyperparameters when searching for a good model.

This does have the potential to make the project quite messy!  Before submitting your project, make sure that you clean up:
- the code you write in this notebook.  The notebook should describe how to train a single CNN-RNN architecture, corresponding to your final choice of hyperparameters.  You should structure the notebook so that the reviewer can replicate your results by running the code in this notebook.  
- the output of the code cell in **Step 2**.  The output should show the output obtained when training the model from scratch.

This notebook **will be graded**.  

Feel free to use the links below to navigate the notebook:
- [Step 1](#step1): Training Setup
- [Step 2](#step2): Train your Model
- [Step 3](#step3): (Optional) Validate your Model

<a id='step1'></a>
## Step 1: Training Setup

In this step of the notebook, you will customize the training of your CNN-RNN model by specifying hyperparameters and setting other options that are important to the training procedure.  The values you set now will be used when training your model in **Step 2** below.

You should only amend blocks of code that are preceded by a `TODO` statement.  **Any code blocks that are not preceded by a `TODO` statement should not be modified**.

### Task #1

Begin by setting the following variables:
- `batch_size` - the batch size of each training batch.  It is the number of image-caption pairs used to amend the model weights in each training step. 
- `vocab_threshold` - the minimum word count threshold.  Note that a larger threshold will result in a smaller vocabulary, whereas a smaller threshold will include rarer words and result in a larger vocabulary.  
- `vocab_from_file` - a Boolean that decides whether to load the vocabulary from file. 
- `embed_size` - the dimensionality of the image and word embeddings.  
- `hidden_size` - the number of features in the hidden state of the RNN decoder.  
- `num_epochs` - the number of epochs to train the model.  We recommend that you set `num_epochs=3`, but feel free to increase or decrease this number as you wish.  [This paper](https://arxiv.org/pdf/1502.03044.pdf) trained a captioning model on a single state-of-the-art GPU for 3 days, but you'll soon see that you can get reasonable results in a matter of a few hours!  (_But of course, if you want your model to compete with current research, you will have to train for much longer._)
- `save_every` - determines how often to save the model weights.  We recommend that you set `save_every=1`, to save the model weights after each epoch.  This way, after the `i`th epoch, the encoder and decoder weights will be saved in the `models/` folder as `encoder-i.pkl` and `decoder-i.pkl`, respectively.
- `print_every` - determines how often to print the batch loss to the Jupyter notebook while training.  Note that you **will not** observe a monotonic decrease in the loss function while training - this is perfectly fine and completely expected!  You are encouraged to keep this at its default value of `100` to avoid clogging the notebook, but feel free to change it.
- `log_file` - the name of the text file containing - for every step - how the loss and perplexity evolved during training.

If you're not sure where to begin to set some of the values above, you can peruse [this paper](https://arxiv.org/pdf/1502.03044.pdf) and [this paper](https://arxiv.org/pdf/1411.4555.pdf) for useful guidance!  **To avoid spending too long on this notebook**, you are encouraged to consult these suggested research papers to obtain a strong initial guess for which hyperparameters are likely to work best.  Then, train a single model, and proceed to the next notebook (**3_Inference.ipynb**).  If you are unhappy with your performance, you can return to this notebook to tweak the hyperparameters (and/or the architecture in **model.py**) and re-train your model.

### Question 1

**Question:** Describe your CNN-RNN architecture in detail.  With this architecture in mind, how did you select the values of the variables in Task 1?  If you consulted a research paper detailing a successful implementation of an image captioning model, please provide the reference.

**Answer:** 


### (Optional) Task #2

Note that we have provided a recommended image transform `transform_train` for pre-processing the training images, but you are welcome (and encouraged!) to modify it as you wish.  When modifying this transform, keep in mind that:
- the images in the dataset have varying heights and widths, and 
- if using a pre-trained model, you must perform the corresponding appropriate normalization.

### Question 2

**Question:** How did you select the transform in `transform_train`?  If you left the transform at its provided value, why do you think that it is a good choice for your CNN architecture?

**Answer:** 

### Task #3

Next, you will specify a Python list containing the learnable parameters of the model.  For instance, if you decide to make all weights in the decoder trainable, but only want to train the weights in the embedding layer of the encoder, then you should set `params` to something like:
```
params = list(decoder.parameters()) + list(encoder.embed.parameters()) 
```

### Question 3

**Question:** How did you select the trainable parameters of your architecture?  Why do you think this is a good choice?

**Answer:** 

### Task #4

Finally, you will select an [optimizer](http://pytorch.org/docs/master/optim.html#torch.optim.Optimizer).

### Question 4

**Question:** How did you select the optimizer used to train your model?

**Answer:** 

In [1]:
import sys
sys.path.append('./cocoapi/PythonAPI')
from pycocotools.coco import COCO
!pip install nltk
import nltk
nltk.download('punkt')
import torch
import torch.nn as nn
from torchvision import transforms
from data_loader import get_loader
from model import EncoderCNN, DecoderRNN
import math


## TODO #1: Select appropriate values for the Python variables below.
batch_size = 3          # batch size
vocab_threshold = 5       # minimum word count threshold
vocab_from_file = True    # if True, load existing vocab file
embed_size = 128          # dimensionality of image and word embeddings
hidden_size = 100        # number of features in hidden state of the RNN decoder
num_epochs = 3             # number of training epochs
save_every = 1             # determines frequency of saving model weights
print_every = 100          # determines window for printing average loss
log_file = 'training_log.txt'       # name of file with saved training loss and perplexity

# (Optional) TODO #2: Amend the image transform below.
transform_train = transforms.Compose([ 
    transforms.Resize(256),                          # smaller edge of image resized to 256
    transforms.RandomCrop(224),                      # get 224x224 crop from random location
    transforms.RandomHorizontalFlip(),               # horizontally flip image with probability=0.5
    transforms.ToTensor(),                           # convert the PIL Image to a tensor
    transforms.Normalize((0.485, 0.456, 0.406),      # normalize image for pre-trained model
                         (0.229, 0.224, 0.225))])

# Build data loader.
data_loader = get_loader(transform=transform_train,
                         mode='train',
                         batch_size=batch_size,
                         vocab_threshold=vocab_threshold,
                         vocab_from_file=vocab_from_file)

# The size of the vocabulary.
vocab_size = len(data_loader.dataset.vocab)

# Initialize the encoder and decoder. 
encoder = EncoderCNN(embed_size)
decoder = DecoderRNN(embed_size, hidden_size, vocab_size)

# Move models to GPU if CUDA is available. 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
encoder.to(device)
decoder.to(device)

# Define the loss function. 
criterion = nn.CrossEntropyLoss().cuda() if torch.cuda.is_available() else nn.CrossEntropyLoss()

# TODO #3: Specify the learnable parameters of the model.
params = decoder.parameters()

# TODO #4: Define the optimizer.
optimizer = torch.optim.Adam(params, lr=0.0001)

# Set the total number of training steps per epoch.
total_step = math.ceil(len(data_loader.dataset.caption_lengths) / data_loader.batch_sampler.batch_size)

You are using pip version 9.0.3, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
[nltk_data] Downloading package punkt to /home/jai/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Vocabulary successfully loaded from vocab.pkl file!
loading annotations into memory...


  0%|          | 0/414113 [00:00<?, ?it/s]

Done (t=0.40s)
creating index...
index created!
Obtaining caption lengths...


100%|██████████| 414113/414113 [00:29<00:00, 14156.99it/s]


<a id='step2'></a>
## Step 2: Train your Model

Once you have executed the code cell in **Step 1**, the training procedure below should run without issue.  

It is completely fine to leave the code cell below as-is without modifications to train your model.  However, if you would like to modify the code used to train the model below, you must ensure that your changes are easily parsed by your reviewer.  In other words, make sure to provide appropriate comments to describe how your code works!  

You may find it useful to load saved weights to resume training.  In that case, note the names of the files containing the encoder and decoder weights that you'd like to load (`encoder_file` and `decoder_file`).  Then you can load the weights by using the lines below:

```python
# Load pre-trained weights before resuming training.
encoder.load_state_dict(torch.load(os.path.join('./models', encoder_file)))
decoder.load_state_dict(torch.load(os.path.join('./models', decoder_file)))
```

While trying out parameters, make sure to take extensive notes and record the settings that you used in your various training runs.  In particular, you don't want to encounter a situation where you've trained a model for several hours but can't remember what settings you used :).

### A Note on Tuning Hyperparameters

To figure out how well your model is doing, you can look at how the training loss and perplexity evolve during training - and for the purposes of this project, you are encouraged to amend the hyperparameters based on this information.  

However, this will not tell you if your model is overfitting to the training data, and, unfortunately, overfitting is a problem that is commonly encountered when training image captioning models.  

For this project, you need not worry about overfitting. **This project does not have strict requirements regarding the performance of your model**, and you just need to demonstrate that your model has learned **_something_** when you generate captions on the test data.  For now, we strongly encourage you to train your model for the suggested 3 epochs without worrying about performance; then, you should immediately transition to the next notebook in the sequence (**3_Inference.ipynb**) to see how your model performs on the test data.  If your model needs to be changed, you can come back to this notebook, amend hyperparameters (if necessary), and re-train the model.

That said, if you would like to go above and beyond in this project, you can read about some approaches to minimizing overfitting in section 4.3.1 of [this paper](http://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=7505636).  In the next (optional) step of this notebook, we provide some guidance for assessing the performance on the validation dataset.

In [3]:
import torch.utils.data as data
import numpy as np
import os
import requests
import time

# Open the training log file.
f = open(log_file, 'w')

old_time = time.time()

for epoch in range(1, num_epochs+1):
    
    for i_step in range(1, total_step+1):
        
        # Randomly sample a caption length, and sample indices with that length.
        indices = data_loader.dataset.get_train_indices()
        # Create and assign a batch sampler to retrieve a batch with the sampled indices.
        new_sampler = data.sampler.SubsetRandomSampler(indices=indices)
        data_loader.batch_sampler.sampler = new_sampler
        
        # Obtain the batch.
        images, captions = next(iter(data_loader))

        # Move batch of images and captions to GPU if CUDA is available.
        images = images.to(device)
        captions = captions.to(device)
        
        # Zero the gradients.
        decoder.zero_grad()
        encoder.zero_grad()
        
        # Pass the inputs through the CNN-RNN model.
        features = encoder(images)
        outputs = decoder(features, captions[:, :-1])
        
        # Calculate the batch loss.
        loss = criterion(outputs.view(-1, vocab_size), captions.view(-1))
        
        # Backward pass.
        loss.backward()
        
        # Update the parameters in the optimizer.
        optimizer.step()
            
        # Get training statistics.
        stats = 'Epoch [%d/%d], Step [%d/%d], Loss: %.4f, Perplexity: %5.4f' % (epoch, num_epochs, i_step, total_step, loss.item(), np.exp(loss.item()))
        
        # Print training statistics (on same line).
        print('\r' + stats, end="")
        sys.stdout.flush()
        
        # Print training statistics to file.
        f.write(stats + '\n')
        f.flush()
        
        # Print training statistics (on different line).
        if i_step % print_every == 0:
            print('\r' + stats)
            
    # Save the weights.
    if epoch % save_every == 0:
        torch.save(decoder.state_dict(), os.path.join('./models', 'decoder-%d.pkl' % epoch))
        torch.save(encoder.state_dict(), os.path.join('./models', 'encoder-%d.pkl' % epoch))

# Close the training log file.
f.close()

Epoch [1/3], Step [100/138038], Loss: 2.1073, Perplexity: 8.22573
Epoch [1/3], Step [200/138038], Loss: 1.8138, Perplexity: 6.133653
Epoch [1/3], Step [300/138038], Loss: 3.0154, Perplexity: 20.3978
Epoch [1/3], Step [400/138038], Loss: 2.0732, Perplexity: 7.950005
Epoch [1/3], Step [500/138038], Loss: 2.2363, Perplexity: 9.358334
Epoch [1/3], Step [600/138038], Loss: 3.2319, Perplexity: 25.32776
Epoch [1/3], Step [700/138038], Loss: 2.9498, Perplexity: 19.1027
Epoch [1/3], Step [800/138038], Loss: 2.7602, Perplexity: 15.8037
Epoch [1/3], Step [900/138038], Loss: 1.7479, Perplexity: 5.74269
Epoch [1/3], Step [1000/138038], Loss: 2.0718, Perplexity: 7.93890
Epoch [1/3], Step [1100/138038], Loss: 3.0420, Perplexity: 20.9478
Epoch [1/3], Step [1200/138038], Loss: 2.5171, Perplexity: 12.3921
Epoch [1/3], Step [1300/138038], Loss: 1.9686, Perplexity: 7.16056
Epoch [1/3], Step [1400/138038], Loss: 2.5132, Perplexity: 12.3443
Epoch [1/3], Step [1500/138038], Loss: 3.2980, Perplexity: 27.0586


Epoch [1/3], Step [12200/138038], Loss: 2.0956, Perplexity: 8.13073
Epoch [1/3], Step [12300/138038], Loss: 3.6374, Perplexity: 37.9922
Epoch [1/3], Step [12400/138038], Loss: 2.1328, Perplexity: 8.438724
Epoch [1/3], Step [12500/138038], Loss: 3.4372, Perplexity: 31.1000
Epoch [1/3], Step [12600/138038], Loss: 3.2756, Perplexity: 26.4595
Epoch [1/3], Step [12700/138038], Loss: 3.0183, Perplexity: 20.4566
Epoch [1/3], Step [12800/138038], Loss: 2.2882, Perplexity: 9.857315
Epoch [1/3], Step [12900/138038], Loss: 2.5864, Perplexity: 13.28180
Epoch [1/3], Step [13000/138038], Loss: 2.2209, Perplexity: 9.21555
Epoch [1/3], Step [13100/138038], Loss: 3.4572, Perplexity: 31.72958
Epoch [1/3], Step [13200/138038], Loss: 2.8030, Perplexity: 16.49360
Epoch [1/3], Step [13300/138038], Loss: 1.9693, Perplexity: 7.16598
Epoch [1/3], Step [13400/138038], Loss: 2.5268, Perplexity: 12.51348
Epoch [1/3], Step [13500/138038], Loss: 2.9461, Perplexity: 19.0319
Epoch [1/3], Step [13600/138038], Loss: 2.

Epoch [1/3], Step [24100/138038], Loss: 2.8817, Perplexity: 17.84521
Epoch [1/3], Step [24200/138038], Loss: 3.3792, Perplexity: 29.34755
Epoch [1/3], Step [24300/138038], Loss: 2.5739, Perplexity: 13.11694
Epoch [1/3], Step [24400/138038], Loss: 2.3819, Perplexity: 10.82539
Epoch [1/3], Step [24500/138038], Loss: 3.8884, Perplexity: 48.83479
Epoch [1/3], Step [24600/138038], Loss: 2.5974, Perplexity: 13.42875
Epoch [1/3], Step [24700/138038], Loss: 3.3128, Perplexity: 27.46327
Epoch [1/3], Step [24800/138038], Loss: 2.7550, Perplexity: 15.72175
Epoch [1/3], Step [24900/138038], Loss: 3.2608, Perplexity: 26.0709
Epoch [1/3], Step [25000/138038], Loss: 2.9898, Perplexity: 19.88090
Epoch [1/3], Step [25100/138038], Loss: 2.1415, Perplexity: 8.51213
Epoch [1/3], Step [25200/138038], Loss: 3.0569, Perplexity: 21.2626
Epoch [1/3], Step [25300/138038], Loss: 3.1833, Perplexity: 24.12686
Epoch [1/3], Step [25400/138038], Loss: 2.4326, Perplexity: 11.38811
Epoch [1/3], Step [25500/138038], Los

Epoch [1/3], Step [36000/138038], Loss: 2.2816, Perplexity: 9.79264
Epoch [1/3], Step [36100/138038], Loss: 2.5006, Perplexity: 12.19005
Epoch [1/3], Step [36200/138038], Loss: 1.8668, Perplexity: 6.46731
Epoch [1/3], Step [36300/138038], Loss: 3.1962, Perplexity: 24.43902
Epoch [1/3], Step [36400/138038], Loss: 3.3298, Perplexity: 27.93414
Epoch [1/3], Step [36500/138038], Loss: 2.1622, Perplexity: 8.69063
Epoch [1/3], Step [36600/138038], Loss: 3.1644, Perplexity: 23.6756
Epoch [1/3], Step [36700/138038], Loss: 2.7430, Perplexity: 15.5332
Epoch [1/3], Step [36800/138038], Loss: 1.8140, Perplexity: 6.134733
Epoch [1/3], Step [36900/138038], Loss: 3.8543, Perplexity: 47.19407
Epoch [1/3], Step [37000/138038], Loss: 3.4390, Perplexity: 31.1562
Epoch [1/3], Step [37100/138038], Loss: 2.2522, Perplexity: 9.50888
Epoch [1/3], Step [37200/138038], Loss: 1.8642, Perplexity: 6.45114
Epoch [1/3], Step [37300/138038], Loss: 2.6673, Perplexity: 14.40172
Epoch [1/3], Step [37400/138038], Loss: 2.

Epoch [1/3], Step [47900/138038], Loss: 3.1202, Perplexity: 22.65009
Epoch [1/3], Step [48000/138038], Loss: 3.8524, Perplexity: 47.1068
Epoch [1/3], Step [48100/138038], Loss: 2.5847, Perplexity: 13.25969
Epoch [1/3], Step [48200/138038], Loss: 2.0853, Perplexity: 8.04739
Epoch [1/3], Step [48300/138038], Loss: 1.9570, Perplexity: 7.078428
Epoch [1/3], Step [48400/138038], Loss: 3.4398, Perplexity: 31.18156
Epoch [1/3], Step [48500/138038], Loss: 3.7587, Perplexity: 42.8926
Epoch [1/3], Step [48600/138038], Loss: 2.0457, Perplexity: 7.73420
Epoch [1/3], Step [48700/138038], Loss: 2.3135, Perplexity: 10.10992
Epoch [1/3], Step [48800/138038], Loss: 2.7472, Perplexity: 15.5997
Epoch [1/3], Step [48900/138038], Loss: 4.0048, Perplexity: 54.8629
Epoch [1/3], Step [49000/138038], Loss: 2.4247, Perplexity: 11.2989
Epoch [1/3], Step [49100/138038], Loss: 1.5961, Perplexity: 4.93354
Epoch [1/3], Step [49200/138038], Loss: 3.0476, Perplexity: 21.06469
Epoch [1/3], Step [49300/138038], Loss: 3.

...

Epoch [3/3], Step [81100/138038], Loss: 3.0732, Perplexity: 21.6102
Epoch [3/3], Step [81200/138038], Loss: 2.5904, Perplexity: 13.3355
Epoch [3/3], Step [81300/138038], Loss: 3.1670, Perplexity: 23.73654
Epoch [3/3], Step [81400/138038], Loss: 2.5190, Perplexity: 12.4164
Epoch [3/3], Step [81500/138038], Loss: 3.3525, Perplexity: 28.5744
Epoch [3/3], Step [81600/138038], Loss: 2.7432, Perplexity: 15.5363
Epoch [3/3], Step [81700/138038], Loss: 2.0731, Perplexity: 7.949456
Epoch [3/3], Step [81800/138038], Loss: 2.4630, Perplexity: 11.74015
Epoch [3/3], Step [81900/138038], Loss: 2.4359, Perplexity: 11.4264
Epoch [3/3], Step [82000/138038], Loss: 1.9464, Perplexity: 7.00370
Epoch [3/3], Step [82100/138038], Loss: 2.2742, Perplexity: 9.720668
Epoch [3/3], Step [82200/138038], Loss: 2.2739, Perplexity: 9.717397
Epoch [3/3], Step [82300/138038], Loss: 3.2490, Perplexity: 25.7657
Epoch [3/3], Step [82400/138038], Loss: 2.0384, Perplexity: 7.67804
Epoch [3/3], Step [82500/138038], Loss: 2.5

Epoch [3/3], Step [93000/138038], Loss: 2.4403, Perplexity: 11.47619
Epoch [3/3], Step [93100/138038], Loss: 2.5799, Perplexity: 13.1962
Epoch [3/3], Step [93200/138038], Loss: 2.4352, Perplexity: 11.4184
Epoch [3/3], Step [93300/138038], Loss: 2.1192, Perplexity: 8.32467
Epoch [3/3], Step [93400/138038], Loss: 2.4789, Perplexity: 11.9277
Epoch [3/3], Step [93500/138038], Loss: 3.2511, Perplexity: 25.8175
Epoch [3/3], Step [93600/138038], Loss: 2.2174, Perplexity: 9.18374
Epoch [3/3], Step [93700/138038], Loss: 2.4181, Perplexity: 11.22443
Epoch [3/3], Step [93800/138038], Loss: 4.1432, Perplexity: 63.00141
Epoch [3/3], Step [93900/138038], Loss: 1.7923, Perplexity: 6.003254
Epoch [3/3], Step [94000/138038], Loss: 2.0700, Perplexity: 7.924554
Epoch [3/3], Step [94100/138038], Loss: 2.7989, Perplexity: 16.4267
Epoch [3/3], Step [94200/138038], Loss: 2.7216, Perplexity: 15.2039
Epoch [3/3], Step [94300/138038], Loss: 1.8444, Perplexity: 6.324674
Epoch [3/3], Step [94400/138038], Loss: 2.

Epoch [3/3], Step [104900/138038], Loss: 2.4330, Perplexity: 11.3932
Epoch [3/3], Step [105000/138038], Loss: 2.9408, Perplexity: 18.9306
Epoch [3/3], Step [105100/138038], Loss: 2.3916, Perplexity: 10.9309
Epoch [3/3], Step [105200/138038], Loss: 2.2373, Perplexity: 9.367648
Epoch [3/3], Step [105300/138038], Loss: 1.9657, Perplexity: 7.13998
Epoch [3/3], Step [105400/138038], Loss: 1.7546, Perplexity: 5.781278
Epoch [3/3], Step [105500/138038], Loss: 3.8104, Perplexity: 45.1678
Epoch [3/3], Step [105600/138038], Loss: 2.5223, Perplexity: 12.4571
Epoch [3/3], Step [105700/138038], Loss: 1.9476, Perplexity: 7.012298
Epoch [3/3], Step [105800/138038], Loss: 2.3949, Perplexity: 10.9671
Epoch [3/3], Step [105900/138038], Loss: 3.0712, Perplexity: 21.5668
Epoch [3/3], Step [106000/138038], Loss: 2.2068, Perplexity: 9.08642
Epoch [3/3], Step [106100/138038], Loss: 1.3863, Perplexity: 4.00023
Epoch [3/3], Step [106200/138038], Loss: 2.1029, Perplexity: 8.18959
Epoch [3/3], Step [106300/13803

Epoch [3/3], Step [116700/138038], Loss: 2.0827, Perplexity: 8.02651
Epoch [3/3], Step [116800/138038], Loss: 2.1352, Perplexity: 8.458641
Epoch [3/3], Step [116900/138038], Loss: 1.8594, Perplexity: 6.42017
Epoch [3/3], Step [117000/138038], Loss: 2.4984, Perplexity: 12.16276
Epoch [3/3], Step [117100/138038], Loss: 2.8563, Perplexity: 17.3973
Epoch [3/3], Step [117200/138038], Loss: 2.4413, Perplexity: 11.4882
Epoch [3/3], Step [117300/138038], Loss: 2.1635, Perplexity: 8.70120
Epoch [3/3], Step [117400/138038], Loss: 3.5980, Perplexity: 36.5235
Epoch [3/3], Step [117500/138038], Loss: 3.8659, Perplexity: 47.7478
Epoch [3/3], Step [117600/138038], Loss: 2.3452, Perplexity: 10.43579
Epoch [3/3], Step [117700/138038], Loss: 2.8199, Perplexity: 16.7744
Epoch [3/3], Step [117800/138038], Loss: 2.2740, Perplexity: 9.71820
Epoch [3/3], Step [117900/138038], Loss: 2.3663, Perplexity: 10.6576
Epoch [3/3], Step [118000/138038], Loss: 2.9567, Perplexity: 19.23494
Epoch [3/3], Step [118100/1380

Epoch [3/3], Step [128500/138038], Loss: 1.9585, Perplexity: 7.08834
Epoch [3/3], Step [128600/138038], Loss: 3.5326, Perplexity: 34.21129
Epoch [3/3], Step [128700/138038], Loss: 1.7996, Perplexity: 6.04749
Epoch [3/3], Step [128800/138038], Loss: 3.5744, Perplexity: 35.6750
Epoch [3/3], Step [128900/138038], Loss: 3.6484, Perplexity: 38.4116
Epoch [3/3], Step [129000/138038], Loss: 2.7261, Perplexity: 15.2739
Epoch [3/3], Step [129100/138038], Loss: 2.2293, Perplexity: 9.29373
Epoch [3/3], Step [129200/138038], Loss: 2.1922, Perplexity: 8.95498
Epoch [3/3], Step [129300/138038], Loss: 2.3897, Perplexity: 10.9103
Epoch [3/3], Step [129400/138038], Loss: 1.9724, Perplexity: 7.18808
Epoch [3/3], Step [129500/138038], Loss: 1.9659, Perplexity: 7.14153
Epoch [3/3], Step [129600/138038], Loss: 3.2038, Perplexity: 24.6251
Epoch [3/3], Step [129700/138038], Loss: 1.3629, Perplexity: 3.90775
Epoch [3/3], Step [129800/138038], Loss: 2.5524, Perplexity: 12.8382
Epoch [3/3], Step [129900/138038]

<a id='step3'></a>
## Step 3: (Optional) Validate your Model

To assess potential overfitting, one approach is to assess performance on a validation set.  If you decide to do this **optional** task, you are required to first complete all of the steps in the next notebook in the sequence (**3_Inference.ipynb**); as part of that notebook, you will write and test code (specifically, the `sample` method in the `DecoderRNN` class) that uses your RNN decoder to generate captions.  That code will prove incredibly useful here. 

If you decide to validate your model, please do not edit the data loader in **data_loader.py**.  Instead, create a new file named **data_loader_val.py** containing the code for obtaining the data loader for the validation data.  You can access:
- the validation images at filepath `'/opt/cocoapi/images/train2014/'`, and
- the validation image caption annotation file at filepath `'/opt/cocoapi/annotations/captions_val2014.json'`.

The suggested approach to validating your model involves creating a json file such as [this one](https://github.com/cocodataset/cocoapi/blob/master/results/captions_val2014_fakecap_results.json) containing your model's predicted captions for the validation images.  Then, you can write your own script or use one that you [find online](https://github.com/tylin/coco-caption) to calculate the BLEU score of your model.  You can read more about the BLEU score, along with other evaluation metrics (such as TEOR and Cider) in section 4.1 of [this paper](https://arxiv.org/pdf/1411.4555.pdf).  For more information about how to use the annotation file, check out the [website](http://cocodataset.org/#download) for the COCO dataset.

In [ ]:
# (Optional) TODO: Validate your model.